In [298]:
import pandas as pd
import os
import datetime
import time
import requests
from bs4 import BeautifulSoup
from PIL import Image
import io
pd.set_option('display.max_columns', None)

class FootballDataScraper:

    def __init__(self, config_file):
        
        self.config_file = config_file
        self.headers = {'User-Agent': 'Mozilla/5.0'}
        self.championnats = self.load_configs()
        self.data_folder = "Database"
        self.logos_folder = "Logos"
        self.futur_data_folder = "Futur_data"
        self.last_request_time = None
        self.request_interval = 3  # secondes
        self.max_seasons = 6

    def load_configs(self):
        # loading config of championship containing which championship are used and their urls
        return pd.read_csv(self.config_file)

    def scrape_or_update(self):
        all_data = {}
        for _, config in self.championnats.iterrows():
            championnat = config['Championnat']
            url = config['URL']
            csv_file_path = os.path.join(self.data_folder, f"{championnat}_data.csv")

            if os.path.exists(csv_file_path):
                data = pd.read_csv(csv_file_path)
                last_update = pd.to_datetime((data['Date'] + ' ' + data['Time']).max())
                if datetime.datetime.now() > last_update + datetime.timedelta(hours = 2):
                    updated_data = self.update_data(url, data)
                    data_updated = updated_data[0].copy()
                    all_data[championnat] = data_updated[data_updated["Comp"]==championnat]
                    futur_data = self.save_futur_data(updated_data[1], championnat)
                    self.save_data(futur_data, os.path.join(self.futur_data_folder, f"{championnat}_futur_data.csv"))
                    
            else:
                data = self.scrape_data(url)
                all_data[championnat] = data[data["Comp"]==championnat]

            self.save_data(all_data[championnat], csv_file_path)
        return all_data

    def scrape_data(self, url):
        
        all_seasons_data = []

        for saison in range(self.max_seasons):

            self.rate_limit()

            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            teams_urls = ["https://fbref.com" + equipe.get("href") 
                        for equipe in soup.select("table.stats_table")[0].find_all("a") 
                        if "squads" in equipe.get("href", "")]

            url = f"https://fbref.com{soup.find('a', class_='button2 prev').get('href')}"

            for team_url in teams_urls:

                self.rate_limit()
                team_response = requests.get(team_url, headers=self.headers)
                try:
                    team_data = pd.read_html(team_response.text, match="Scores")[0]
                except ValueError as e:
                    # Gestion de l'exception si aucune table n'est trouvée
                    print(f"Aucune table trouvée dans {team_url} - Erreur: {e}")
                    continue  # Passe à l'itération suivante de la boucle

                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                team_data["Team"] = team_name

                self.scrape_and_save_logo(team_url, team_name)

                # Récupération des URLs pour les statistiques détaillées
                url_stats = {
                    f"https://fbref.com{a.get('href')}" 
                    for a in BeautifulSoup(team_response.text, 'html.parser').find_all("a") 
                    if "matchlogs/all_comps" in a.get('href', '') and 
                    any(substring in a.get('href', '') for substring in ["passing/", "shooting", "possession/", "defense/", "keeper"])
                }

                # Traitement des statistiques détaillées
                for stats_url in url_stats:

                    self.rate_limit()

                    stats_response = requests.get(stats_url, headers=self.headers)

                    try:
                        detailed_stats = pd.read_html(stats_response.text)[0]
                    except ValueError as e:
                        print(f"Aucune table trouvée dans {stats_url} - Erreur: {e}")
                        continue  # Passe à l'itération suivante de la boucle

                    # Nettoyage des colonnes du DataFrame
                    if detailed_stats.columns.nlevels > 1:
                        detailed_stats.columns = [f"{col}_{branch}" 
                                                if "For" not in col and "Unnamed:" not in col 
                                                else f"{branch}" 
                                                for col, branch in detailed_stats.columns]

                    columns_to_drop = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"] + [col for col in detailed_stats.columns if 'Report' in col]
                    columns_to_drop = [col for col in columns_to_drop if col in detailed_stats.columns]

                    detailed_stats.drop(columns_to_drop, axis=1, inplace=True)

                    team_data = team_data.merge(detailed_stats, on="Date")

                # Ajout des données de l'équipe au résultat global
                all_seasons_data.append(team_data)

        return pd.concat(all_seasons_data, ignore_index=True)

    def update_data(self, url, data):

        futur_data = []
        all_data = []
        response = requests.get(url, headers=self.headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        teams_urls = ["https://fbref.com" + equipe.get("href") 
                    for equipe in soup.select("table.stats_table")[0].find_all("a") 
                    if "squads" in equipe.get("href", "")]

        # Traitement similaire à scrape_ligue1_data() pour chaque équipe
        for team_url in teams_urls:

            self.rate_limit()

            team_response = requests.get(team_url, headers=self.headers)
            try:
                team_data = pd.read_html(team_response.text, match="Scores")[0]
            except ValueError as e:
                # Gestion de l'exception si aucune table n'est trouvée
                print(f"Aucune table trouvée dans {team_url} - Erreur: {e}")
                continue  # Passe à l'itération suivante de la boucle

            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            team_data["Team"] = team_name

            futur_data.append(team_data)


            # Récupération des URLs pour les statistiques détaillées
            url_stats = {
                f"https://fbref.com{a.get('href')}" 
                for a in BeautifulSoup(team_response.text, 'html.parser').find_all("a") 
                if "matchlogs/all_comps" in a.get('href', '') and 
                any(substring in a.get('href', '') for substring in ["passing/", "shooting", "possession/", "defense/", "keeper"])
            }

            for stats_url in url_stats:

                self.rate_limit()

                stats_response = requests.get(stats_url, headers=self.headers)
                
                try:
                    detailed_stats = pd.read_html(stats_response.text)[0]
                except ValueError as e:
                    print(f"Aucune table trouvée dans {stats_url} - Erreur: {e}")
                    continue  # Passe à l'itération suivante de la boucle


                # Nettoyage des colonnes du DataFrame
                if detailed_stats.columns.nlevels > 1:
                    detailed_stats.columns = [f"{col}_{branch}" 
                                            if "For" not in col and "Unnamed:" not in col 
                                            else f"{branch}" 
                                            for col, branch in detailed_stats.columns]

                columns_to_drop = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"] + [col for col in detailed_stats.columns if 'Report' in col]
                columns_to_drop = [col for col in columns_to_drop if col in detailed_stats.columns]

                detailed_stats.drop(columns_to_drop, axis=1, inplace=True)

                team_data = team_data.merge(detailed_stats, on="Date")

            # Ajout des données de l'équipe au résultat global
            all_data.append(team_data)

        new_data = pd.concat(all_data, ignore_index=True)

        # Concaténation de la base initiale et de la base nouvelle
        concatenated_df = pd.concat([data, new_data])
        
        # Suppression des doublons basée sur les colonnes "Date", "Team" et "Opponent"
        concatenated_df = concatenated_df.drop_duplicates(subset=['Date', 'Team', 'Opponent'])
            
        return concatenated_df.reset_index(drop=True), pd.concat(futur_data, ignore_index=True)

    def save_data(self, data, file_path):
        # Enregistrer les données dans un fichier CSV
        data.to_csv(file_path, index=False)

    def rate_limit(self):
        if self.last_request_time is not None:
            elapsed_time = time.time() - self.last_request_time
            if elapsed_time < self.request_interval:
                time.sleep(self.request_interval - elapsed_time)
        self.last_request_time = time.time()

    def access_data(self, championnat):
        csv_file_path = os.path.join(self.data_folder, f"{championnat}_data.csv")
        
        if os.path.exists(csv_file_path):
            data = pd.read_csv(csv_file_path)
            return data
        else:
            print(f"Les données pour le championnat '{championnat}' n'existent pas.")

    def scrape_and_save_logo(self, page_url, file_name):

        file_path = os.path.join(self.logos_folder, file_name + '.png')
        if os.path.exists(file_path):
            print(f"Le logo existe déjà à {file_path}")
            return

        try:
            self.rate_limit()
            response = requests.get(page_url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            logo_img = soup.find('img', {'class': 'teamlogo'})

            if logo_img and logo_img.get('src'):
                logo_url = logo_img['src']
                self.rate_limit()
                img_response = requests.get(logo_url)

                if img_response.status_code == 200:
                    # Convertir l'image en PNG
                    image = Image.open(io.BytesIO(img_response.content))
                    file_path = os.path.join(self.logos_folder, file_name)
                    if not file_path.endswith('.png'):
                        file_path += '.png'
                    image.save(file_path, 'PNG')
                    print(f"Logo sauvegardé en PNG à {file_path}")
                else:
                    print("Erreur lors du téléchargement de l'image")
            else:
                print("Logo non trouvé sur la page")

        except Exception as e:
            print(f"Une erreur s'est produite : {e}")

    def save_futur_data(self, data, championnat):

        """
        À partir du scrapping on récupère un DataFrame qui contient les futurs journées, il faut le mettre en forme.
        args: DataFrame, le mapping ligue1
        """

        # Supprimer les lignes où les colonnes 'Date', 'Time' et 'Round' sont manquantes.
        data.dropna(subset=["Date", "Time", "Round"], inplace=True)
        data['DateTime'] = pd.to_datetime((data['Date'] + ' ' + data['Time']))
        data = data[data["Comp"] == championnat]

        data = data[data['DateTime'] >= datetime.datetime.now()].sort_values(by="DateTime")


        if data.empty != True:
                first_date = data['DateTime'].iloc[0]
        else:
            return None
        
        ten_days = datetime.timedelta(days=10) + first_date

        # Filtrer pour garder seulement les matchs programmés dans les 10 jours suivant la 'premiere_date_proche'.
        data = data[data['DateTime'] <= ten_days]

        if data.empty == True:
            return None
        else:# Retourner le DataFrame s'il n'est pas vide, sinon retourner None.
            csv_file_path = os.path.join(self.futur_data_folder, f"{championnat}_futur_data.csv")
            if os.path.exists(csv_file_path):
                futur_data = pd.read_csv(csv_file_path)
                last_update = pd.to_datetime((futur_data['Date'] + ' ' + futur_data['Time']).max())
                if last_update == pd.to_datetime((data['Date'] + ' ' + data['Time']).max()):
                    return futur_data
                else:
                    return pd.concat([futur_data, data]).drop_duplicates(subset=["Team", "Opponent", "Date"])
            else:
                return data

            

    






        


    









In [11]:
allo = FootballDataScraper("config.csv")
allo.scrape_or_update()

/tmp/ipykernel_207300/1508494673.py:143: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]
/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a '

Aucune table trouvée dans https://fbref.com/en/squads/7a41008f/2023-2024/matchlogs/all_comps/defense/Union-Berlin-Match-Logs-All-Competitions - Erreur: No tables found


/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_207300/1508494673.py:170: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' 

{'La Liga':             Date   Time     Comp         Round  Day Venue Result   GF   GA  \
 0     2023-08-12  17:00  La Liga   Matchweek 1  Sat  Away      D  1.0  1.0   
 1     2023-08-20  19:00  La Liga   Matchweek 2  Sun  Home      W  3.0  0.0   
 2     2023-08-26  21:30  La Liga   Matchweek 3  Sat  Away      W  2.0  1.0   
 3     2023-09-03  14:00  La Liga   Matchweek 4  Sun  Home      W  1.0  0.0   
 4     2023-09-18  21:00  La Liga   Matchweek 5  Mon  Away      W  4.0  2.0   
 ...          ...    ...      ...           ...  ...   ...    ...  ...  ...   
 5085  2024-01-21  14:00  La Liga  Matchweek 21  Sun  Home      W  3.0  2.0   
 5086  2024-01-21  21:00  La Liga  Matchweek 21  Sun  Away      L    1    5   
 5087  2024-01-21  16:15  La Liga  Matchweek 21  Sun  Away      L  2.0  3.0   
 5088  2024-01-22  21:00  La Liga  Matchweek 21  Mon  Away      W  1.0  0.0   
 5089  2024-01-22  21:00  La Liga  Matchweek 21  Mon  Home      L  0.0  1.0   
 
              Opponent  ...  Challenges

In [301]:

from Levenshtein import distance as levenshtein_distance
import json

class Football_viz:

    def __init__(self):
        
        self.data_folder = "Database"
        self.logos_folder = "Logos"
        self.futur_data_folder = "Futur_data"
        self.mapping = 'mapping_team.json'

    def read_data(self, championnat):
        data = self._read_raw_data(championnat)
        data = self.mapped_data(championnat, data)
        data = self.processing_data(data, championnat)
        return data
        
    def _read_raw_data(self, championnat):
        path = os.path.join(self.data_folder, f"{championnat}_data.csv")
        data = pd.read_csv(path)
        return data
    
    def read_all_data(self):
        list_data = []
        for fichier in os.listdir(self.data_folder):
            if fichier.endswith('.csv'):
                path = os.path.join(self.data_folder, fichier)
                data = pd.read_csv(path)
                list_data.append(data)
        
        return pd.concat(list_data, ignore_index=True)

    def find_mapping(self, championnat):
        # show the difference of team name between the two columns
        # help to modify the file for mapping

        data = self.read_data(championnat)
        problem_names_1 = set(data['Team'].unique()) - set(data['Opponent'].unique())
        problem_names_2 = set(data['Opponent'].unique()) - set(data['Team'].unique())

        print(sorted(problem_names_1))
        print(sorted(problem_names_2))

    def mapped_data(self, championnat, data):

        with open(self.mapping, 'r', encoding='utf-8') as file:
            mappings = json.load(file)
        
        mapping = mappings.get(championnat, {})

        data['Opponent'] = data['Opponent'].map(mapping).fillna(data['Opponent'])
        data['Team'] = data['Team'].map(mapping).fillna(data['Team'])

        return data

    def processing_data(self, data, championnat):
        
        data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])
        data = data[data["Comp"] == championnat]

        try:
            data['Round'] = data['Round'].str.extract(r'(\d+)').astype(int)
        except (ValueError, TypeError):
            data = data.iloc[1:]
            
        data['Season'] = data['DateTime'].apply(lambda x: f"{x.year}-{x.year + 1}" if x.month >= 8 else f"{x.year - 1}-{x.year}")
        data['Formation'] = data['Formation'].apply(lambda x: x.replace('◆', '') if pd.notnull(x) else x)
        data[['GF', 'GA']] = data[['GF', 'GA']].astype(float).astype(int)
        data['GD'] = data['GF'] - data['GA']
        data['Points'] = data['Result'].map({'W': 3, 'D': 1, 'L': 0})


        data.sort_values(by=['Season', 'Round', 'Team'], inplace=True)
        data.reset_index(drop=True, inplace=True)

        cumulative_cols = data.groupby(['Season', 'Team']).agg({
            'Points': 'cumsum',
            'GD': 'cumsum',
            'GF': 'cumsum',
            'GA': 'cumsum'
        }).reset_index()

        data[['Points_Cum', 'GD_Cum', 'GF_Cum', 'GA_Cum']] = cumulative_cols[['Points', 'GD', 'GF', 'GA']]


        data.sort_values(by=['Season', 'Round', 'Points_Cum', 'GD_Cum'], ascending=[True, True, False, False], inplace=True)
        data['Classement'] = data.groupby(['Season', 'Round']).cumcount() + 1

        data["Total_Goals"] = data["GF"] + data["GA"]
        data["Minus 1.5 Goals"] = (data["Total_Goals"] <= 1.5).astype(int)
        data["Minus 2.5 Goals"] = (data["Total_Goals"] <= 2.5).astype(int)
        data["Minus 3.5 Goals"] = (data["Total_Goals"] <= 3.5).astype(int)

        return data



In [348]:
foot = Football_viz()
a = foot._read_raw_data('Serie A')

In [352]:
b = foot.read_data('Eredivisie')

ValueError: cannot convert float NaN to integer

In [350]:
b['Round'].unique()

array(['Matchweek 1', 'Matchweek 2', 'Matchweek 4', 'Matchweek 5',
       'Matchweek 6', 'Matchweek 3', 'Matchweek 7', 'Matchweek 8',
       'Matchweek 9', 'Matchweek 10', 'Matchweek 11', 'Matchweek 12',
       'Matchweek 13', 'Matchweek 14', 'Matchweek 15', 'Matchweek 16',
       'Matchweek 17', 'Matchweek 18', 'Matchweek 19', 'Matchweek 20',
       'Matchweek 21', 'Matchweek 22', 'Matchweek 23', 'Matchweek 24',
       'Matchweek 25', 'Matchweek 26', 'Matchweek 27', 'Matchweek 28',
       'Matchweek 29', 'Matchweek 30', 'Matchweek 31', 'Matchweek 32',
       'Matchweek 33', 'Matchweek 34', 'Semi-finals', 'Finals',
       'Regular season'], dtype=object)

In [351]:
b[b['Round']=='Finals']

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err
494,2023-06-08,20:00,Eredivisie,Finals,Thu,Away,D,1,1,Sparta R'dam,1.5,1.3,63.0,10349.0,Robin Pröpper,4-2-3-1,Jeroen Manschot,Match Report,Leg 1 of 2,Twente,671.0,46.0,168.0,283.0,227.0,36.0,671.0,11.0,5.0,45.5,6.0,54.5,443.0,2790.0,1538.0,27.0,20.0,7.0,17.0,7.0,463.0,56.0,2.0,1.0,1.0,100.0,0.0,1.3,0.3,1.0,1.0,0.0,0.0,4.0,11.0,36.4,32.0,8.0,28.1,30.1,7.0,28.6,37.7,10.0,1.0,10.0,3.0,20.8,1,19.0,3.0,15.8,0.05,0.33,18.5,0.0,0,0,1.5,1.5,0.08,-0.5,-0.5,472.0,583.0,81.0,8325.0,2810.0,222.0,240.0,92.5,188.0,222.0,84.7,46.0,84.0,54.8,1,1.0,1.7,12.0,48.0,16.0,4.0,57.0,9.0,7.0,4.0,0.0,5.0,4.0,12.0,33.3,8.0,6.0,3.0,3.0,8.0,17,7.0,0.0
495,2023-06-11,14:30,Eredivisie,Finals,Sun,Home,W,1,0,Sparta R'dam,1.9,0.5,60.0,29500.0,Wout Brama,4-2-3-1,Danny Makkelie,Match Report,Leg 2 of 2; Twente won,Twente,679.0,51.0,121.0,302.0,265.0,46.0,679.0,30.0,15.0,50.0,12.0,40.0,395.0,2194.0,1207.0,29.0,18.0,10.0,13.0,15.0,438.0,51.0,3.0,0.0,3.0,100.0,1.0,0.4,0.4,0.0,0.0,0.0,0.0,2.0,8.0,25.0,20.0,4.0,30.0,34.6,4.0,50.0,38.8,11.0,1.0,9.1,3.0,17.6,1,25.0,9.0,36.0,0.04,0.11,18.3,0.0,0,0,1.9,1.9,0.08,-0.9,-0.9,451.0,564.0,80.0,7932.0,2661.0,213.0,241.0,88.4,186.0,213.0,87.3,43.0,82.0,52.4,1,1.5,2.4,20.0,35.0,15.0,3.0,51.0,8.0,6.0,5.0,1.0,2.0,6.0,16.0,37.5,10.0,7.0,1.0,6.0,5.0,13,17.0,0.0
532,2023-06-08,20:00,Eredivisie,Finals,Thu,Home,D,1,1,Twente,1.3,1.5,37.0,10349.0,Bart Vriends,4-3-3,Jeroen Manschot,Match Report,Leg 1 of 2,Sparta Rotterdam,445.0,82.0,185.0,177.0,88.0,18.0,444.0,17.0,8.0,47.1,4.0,23.5,254.0,1410.0,604.0,11.0,5.0,4.0,14.0,5.0,231.0,17.0,3.0,1.0,2.0,66.7,0.0,0.8,-0.2,0.0,0.0,0.0,0.0,19.0,38.0,50.0,39.0,1.0,66.7,47.4,15.0,80.0,57.5,13.0,1.0,7.7,0.0,11.8,1,8.0,1.0,12.5,0.00,0.00,20.6,0.0,1,1,1.3,0.6,0.07,-0.3,-0.6,233.0,339.0,68.7,4836.0,2435.0,96.0,125.0,76.8,91.0,116.0,78.4,37.0,77.0,48.1,0,0.5,0.7,6.0,21.0,2.0,0.0,17.0,13.0,6.0,8.0,3.0,2.0,6.0,11.0,54.5,5.0,14.0,7.0,7.0,6.0,19,20.0,0.0
533,2023-06-11,14:30,Eredivisie,Finals,Sun,Away,L,0,1,Twente,0.5,1.9,40.0,29500.0,Bart Vriends,4-3-3,Danny Makkelie,Match Report,Leg 2 of 2; Twente won,Sparta Rotterdam,523.0,104.0,257.0,189.0,81.0,17.0,523.0,19.0,10.0,52.6,6.0,31.6,230.0,1505.0,627.0,10.0,11.0,1.0,18.0,2.0,267.0,17.0,9.0,1.0,8.0,88.9,0.0,1.3,0.3,0.0,0.0,0.0,0.0,14.0,24.0,58.3,37.0,5.0,54.1,40.6,9.0,44.4,38.1,20.0,1.0,5.0,1.0,12.5,0,9.0,3.0,33.3,0.00,0.00,15.7,1.0,0,0,0.5,0.5,0.06,-0.5,-0.5,269.0,375.0,71.7,5358.0,2279.0,119.0,148.0,80.4,110.0,134.0,82.1,39.0,79.0,49.4,0,0.4,0.4,6.0,11.0,5.0,1.0,17.0,27.0,16.0,18.0,6.0,3.0,12.0,27.0,44.4,15.0,21.0,11.0,10.0,2.0,29,32.0,0.0
1118,2022-05

In [320]:
a['Date'] = pd.to_datetime(a['Date'])

In [330]:
a[a['Round'] == 'Relegation tie-breaker']

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist
1050,2023-06-11,20:45,Serie A,Relegation tie-breaker,Sun,Home,L,1.0,3.0,Hellas Verona,1.9,0.8,68.0,NaN,Dimitris Nikolaou,3-5-2,Daniele Orsato,Match Report,NaN,Spezia,419.0,537.0,78.0,7689.0,2805.0,177.0,199.0,88.9,176.0,213.0,82.6,57.0,103.0,55.3,1.0,0.7,0.8,8.0,35.0,6.0,5.0,36.0,8.0,6.0,4.0,4.0,0.0,5.0,12.0,41.7,7.0,6.0,2.0,4.0,5.0,13.0,19.0,0.0,1.0,16.0,8.0,50.0,0.06,0.13,18.0,1.0,0.0,1.0,1.9,1.4,0.09,-0.9,-0.4,646.0,57.0,211.0,251.0,188.0,25.0,645.0,26.0,9.0,34.6,14.0,53.8,360.0,2042.0,1021.0,24.0,17.0,3.0,20.0,10.0,417.0,36.0,5.0,3.0,2.0,40.0,0.0,0.8,-2.2,0.0,0.0,0.0,0.0,6.0,16.0,37.5,35.0,7.0,40.0,36.4,5.0,40.0,40.0,8.0,1.0,12.5,0.0,10.0
1089,2023-06-11,20:45,Serie A,Relegation tie-breaker,Sun,Away,W,3.0,1.0,Spezia,0.8,1.9,32.0,NaN,Davide Faraoni,3-4-3,Daniele Orsato,Match Report,NaN,Hellas Verona,142.0,250.0,56.8,3324.0,1948.0,54.0,76.0,71.1,52.0,75.0,69.3,33.0,84.0,39.3,2.0,0.6,0.8,8.0,16.0,6.0,0.0,16.0,24.0,16.0,14.0,9.0,1.0,14.0,23.0,60.9,9.0,5.0,1.0,4.0,6.0,30.0,39.0,0.0,3.0,9.0,5.0,55.6,0.33,0.60,16.8,0.0,0.0,0.0,0.8,0.8,0.08,2.2,2.2,373.0,72.0,168.0,125.0,86.0,18.0,373.0,14.0,7.0,50.0,5.0,35.7,130.0,644.0,321.0,6.0,6.0,1.0,14.0,3.0,137.0,16.0,9.0,1.0,7.0,88.9,0.0,1.9,0.9,1.0,0.0,1.0,0.0,14.0,32.0,43.8,23.0,1.0,73.9,53.0,15.0,100.0,71.8,30.0,1.0,3.3,1.0,12.0


In [323]:
a.sort_values(by='Date', ascending=False)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist
4175,2024-01-21,18:00,Serie A,Matchweek 21,Sun,Home,L,1.0,2.0,Genoa,0.8,1.1,60.0,NaN,Antonio Candreva,4-3-2-1,Daniele Orsato,Match Report,NaN,Salernitana,470.0,596.0,78.9,8397.0,2647.0,194.0,220.0,88.2,210.0,247.0,85.0,49.0,84.0,58.3,1.0,0.5,0.7,8.0,31.0,8.0,3.0,37.0,18.0,15.0,8.0,7.0,3.0,7.0,9.0,77.8,2.0,18.0,0.0,18.0,11.0,29.0,13.0,0.0,1.0,11.0,2.0,18.2,0.09,0.50,16.6,1.0,0.0,0.0,0.8,0.8,0.07,0.2,0.2,703.0,51.0,176.0,360.0,172.0,17.0,703.0,17.0,8.0,47.1,8.0,47.1,412.0,1856.0,919.0,12.0,11.0,2.0,12.0,3.0,469.0,37.0,5.0,2.0,3.0,80.0,0.0,1.9,-0.1,1.0,1.0,0.0,0.0,4.0,12.0,33.3,29.0,4.0,34.5,36.2,2.0,100.0,69.0,8.0,1.0,12.5,1.0,23.3
4174,2024-01-21,15:00,Serie A,Matchweek 21,Sun,Home,W,3.0,0.0,Monza,1.3,0.8,28.0,NaN,Sebastiano Luperto,3-5-2,Antonio Giua,Match Report,NaN,Empoli,196.0,284.0,69.0,3981.0,1730.0,73.0,94.0,77.7,103.0,124.0,83.1,20.0,58.0,34.5,1.0,1.2,0.6,6.0,12.0,4.0,0.0,23.0,15.0,8.0,10.0,5.0,0.0,3.0,7.0,42.9,4.0,10.0,5.0,5.0,8.0,23.0,41.0,0.0,3.0,9.0,5.0,55.6,0.33,0.60,16.2,0.0,0.0,0.0,1.3,1.3,0.15,1.7,1.7,388.0,61.0,183.0,151.0,61.0,14.0,388.0,8.0,1.0,12.5,6.0,75.0,195.0,1004.0,455.0,11.0,6.0,2.0,7.0,7.0,195.0,23.0,5.0,0.0,5.0,100.0,1.0,1.4,1.4,0.0,0.0,0.0,0.0,5.0,17.0,29.4,21.0,2.0,57.1,46.3,7.0,71.4,57.9,22.0,1.0,4.5,1.0,17.0
4168,2024-01-21,20:45,Serie A,Matchweek 21,Sun,Away,W,3.0,0.0,Lecce,2.1,0.6,49.0,NaN,Danilo,3-5-2,Daniele Doveri,Match Report,NaN,Juventus,387.0,482.0,80.3,7115.0,2752.0,162.0,187.0,86.6,167.0,191.0,87.4,48.0,80.0,60.0,3.0,2.3,1.2,12.0,36.0,14.0,4.0,41.0,16.0,9.0,7.0,8.0,1.0,2.0,8.0,25.0,6.0,11.0,5.0,6.0,7.0,23.0,21.0,0.0,3.0,15.0,6.0,40.0,0.20,0.50,11.3,0.0,0.0,0.0,2.1,2.1,0.17,0.9,0.9,592.0,68.0,204.0,256.0,140.0,28.0,592.0,13.0,8.0,61.5,5.0,38.5,318.0,1614.0,804.0,16.0,10.0,2.0,16.0,11.0,383.0,41.0,3.0,0.0,3.0,100.0,1.0,0.9,0.9,0.0,0.0,0.0,0.0,6.0,13.0,46.2,27.0,6.0,37.0,33.6,9.0,33.3,31.3,17.0,0.0,0.0,0.0,10.0
4169,2024-01-21,18:00,Serie A,Matchweek 21,Sun,Away,W,2.0,1.0,Salernitana,1.1,0.8,40.0,NaN,Milan Badelj,4-4-2,Daniele Orsato,Match Report,NaN,Genoa,288.0,398.0,72.4,5329.0,2407.0,136.0,168.0,81.0,114.0,130.0,87.7,33.0,67.0,49.3,1.0,0.3,0.4,3.0,24.0,5.0,0.0,30.0,11.0,8.0,7.0,4.0,0.0,8.0,16.0,50.0,8.0,17.0,6.0,11.0,12.0,23.0,31.0,0.0,2.0,4.0,4.0,100.0,0.25,0.25,19.0,0.0,1.0,1.0,1.1,0.4,0.09,0.9,0.6,509.0,66.0,197.0,213.0,103.0,17.0,508.0,9.0,2.0,22.2,7.0,77.8,250.0,1109.0,540.0,11.0,12.0,4.0,8.0,11.0,283.0,28.0,2.0,1.0,1.0,50.0,0.0,1.1,0.1,0.0,0.0,0.0,0.0,11.0,21.0,52.4,32.0,1.0,53.1,41.4,6.0,66.7,45.8,20.0,0.0,0.0,1.0,11.2
4170,2024-01-21,15:00,Serie A,Matchweek 21,Sun,Away